In [6]:
import pandas as pd
import os
import urllib
import pdb
import cv2
import numpy as np
from matplotlib import pyplot as plt

IMAGE_ROOT = './image'
MASK_ROOT = './mask'
def mkMask(img_root, mask_root):
    garbage = []
    back_image = cv2.imread('./background.jpg')
    hsv = cv2.cvtColor(back_image,cv2.COLOR_BGR2HSV)
    
    for idx, img_name in enumerate(os.listdir(img_root)):
        path = os.path.join(img_root, img_name)
        path2 = os.path.join(mask_root, img_name)
        img = cv2.imread(path)
        
        target = cv2.imread(path)
        #hsvt = cv2.cvtColor(target,cv2.COLOR_BGR2HSV)
        
        try:
            hsvt = cv2.cvtColor(target,cv2.COLOR_BGR2HSV)
        except Exception as e:
            print(img_name)
            garbage.append(img_name)
            #print(garbage)
            continue
        
        
        M = cv2.calcHist([hsv],[0, 1], None, [180, 256], [0, 180, 0, 256] )
        I = cv2.calcHist([hsvt],[0, 1], None, [180, 256], [0, 180, 0, 256] )
        R = M/(I+1)
        
        h,s,v = cv2.split(hsvt)
        B = R[h.ravel(),s.ravel()]
        B = np.minimum(B,1)
        B = B.reshape(hsvt.shape[:2])
        
        disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
        cv2.filter2D(B,-1,disc,B)
        B = np.uint8(B)
        cv2.normalize(B,B,0,255,cv2.NORM_MINMAX)
        ret,thresh = cv2.threshold(B,50,255,0)
        thresh = cv2.merge((thresh,thresh,thresh))
        thresh = cv2.bitwise_not(thresh)
        res = cv2.bitwise_and(target,thresh)
        
        cv2.imwrite(path2, res)
        


In [7]:
mkMask(IMAGE_ROOT, MASK_ROOT)

image.zip
.ipynb_checkpoints
